# Generate sample size for CONSORT figures
- ABCD
- BANDA
- also outputs list of final ids that were included in both datasets 

In [1]:
import pandas as pd

In [2]:
main_dir = '/projects/swglab/data/BANDA_U01/analysis/proj_franci_anxiety'

# ABCD
considering ids as the dynamic list of included subject

grab ids of those that have some fmridata (belowthresh)

In [3]:
pconn_ids = pd.read_csv(f'{main_dir}/cpm/abcd_dcan/belowthresh/connectomes/pconn_ids.csv', sep=',')
mri_ids = pconn_ids['participant_id'].to_list()
mri_ids = ['sub-' + x for x in mri_ids]
len(mri_ids)

9404

cbcl

In [4]:
cbcl = pd.read_csv(f'{main_dir}/behave_inputs/abcd_dcan/abcd_dcan_allsubjs_belowthresh.csv', sep = ",")
cbcl = cbcl.set_index('participant_id')
len(cbcl)

9404

drop ids

In [5]:
ids = mri_ids.copy()
dropids=['sub-NDARINVP2G0PXCM'] #is in Collection 3165 but miss all info from ABCD. in CONSORT figure count towards: missing cbcl base

for i in dropids:
    ids.remove(i)

len(ids)

9403

In [6]:
cbcl = cbcl.loc[ids]
print(len(cbcl))

9403


cbcl

In [7]:
cbcl = pd.read_csv(f'{main_dir}/behave_inputs/abcd_dcan/abcd_dcan_allsubjs_belowthresh.csv', sep = ",")
cbcl = cbcl.set_index('participant_id')
cbcl = cbcl.loc[ids]

print(len(cbcl))

9403


In [8]:
#consort cbcl baseline
newid = cbcl[cbcl['cbcl_scr_syn_anxdep_t_bl'].notna()].index 
cbcl = cbcl.loc[newid]
print(len(cbcl))

9400


In [9]:
#consort cbcl 1 year followup
newid = cbcl[cbcl['cbcl_scr_syn_anxdep_t_y1'].notna()].index 
cbcl = cbcl.loc[newid]
len(cbcl)

8889

In [10]:
ids = newid.copy().to_list()
len(ids)

8889

grab info about site

In [11]:
site = pd.read_csv(f'{main_dir}/data/abcd/ABCDStudyNDA/abcd_lt01.txt', sep = "\t", skiprows=[1])
site['participant_id'] = site['src_subject_id'].str.replace('_','').str.replace('NDAR','sub-NDAR')
site = site.set_index('participant_id')
site['site']=site['site_id_l']
site = site[site['eventname'] == 'baseline_year_1_arm_1']
site = site.loc[ids]

site[site['site'].notna()].shape

(8889, 15)

family

In [12]:
fam = pd.read_csv(f'{main_dir}/data/abcd/ABCDStudyNDA/acspsw03.txt', sep = "\t", skiprows=[1])
fam['participant_id'] = fam['src_subject_id'].str.replace('_','').str.replace('NDAR','sub-NDAR')
fam = fam.set_index('participant_id')
fam = fam[fam['eventname'] == 'baseline_year_1_arm_1']
fam = fam.loc[ids]

len(fam[fam['rel_family_id'].notna()])

8889

check if among all non-only child
site is different based on rel_family_id 

In [13]:
df = pd.merge(fam, site, on='participant_id')
tmp = pd.crosstab(df['rel_family_id'], df['site']) # cross tab of where each family was scanned. this is 1s and 0s
tmp['nonzero'] = tmp.astype(bool).sum(axis=1) # generating sum by row = by familyid
tmp = tmp[tmp['nonzero']!=1] # if the sum is different from 1 then at least one family member of the same family was scanned at two sites
tmp #these are the families for which sibling/twin were scanned at different sites

site,site01,site02,site03,site04,site05,site06,site07,site08,site09,site10,...,site14,site15,site16,site17,site18,site19,site20,site21,site22,nonzero
rel_family_id,,,,,,,,,,,,,,,,,,,,,
3764.0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,2
6880.0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
6887.0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,2
9942.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,2
11619.0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2


In [14]:
len(tmp) # this is how many families had at least one memeber that was scanned at another site

5

In [15]:
# explore
#gives back the site corresponding to the family memeber of tmp.index[0] which is eg 3764.0
for i in tmp.index:
    print(df[['rel_family_id','site']][df['rel_family_id'] == i])
    print('\n')

                     rel_family_id    site
participant_id                            
sub-NDARINVB3CU3GHK         3764.0  site16
sub-NDARINVXLFHB010         3764.0  site14


                     rel_family_id    site
participant_id                            
sub-NDARINVG19M2F39         6880.0  site08
sub-NDARINVZRABEXLZ         6880.0  site06


                     rel_family_id    site
participant_id                            
sub-NDARINVGVPPRTDN         6887.0  site08
sub-NDARINVVH9GWJVF         6887.0  site20


                     rel_family_id    site
participant_id                            
sub-NDARINVC2ZLETPN         9942.0  site21
sub-NDARINVZZL0VA2F         9942.0  site22


                     rel_family_id    site
participant_id                            
sub-NDARINVDJDNYY46        11619.0  site17
sub-NDARINVGFCRX7YW        11619.0  site02




In [16]:
# df[df['rel_family_id'].isin(tmp.index)][['site','rel_relationship','rel_family_id','scanner_manufacturer', 'scanner_model', 'scanner_software','cbcl_scr_syn_anxdep_r_bl','cbcl_scr_syn_anxdep_r_y1','sex','interview_age_bl']].sort_values(by='rel_family_id')

In [17]:
print(df.shape)
df['site'].value_counts()

(8889, 49)


site
site16    915
site20    593
site04    577
site14    545
site03    517
site06    509
site02    488
site21    481
site17    475
site13    458
site19    450
site10    449
site09    368
site15    359
site11    358
site05    318
site07    267
site18    257
site08    246
site12    147
site01     95
site22     17
Name: count, dtype: int64

In [18]:
# drop ids of one of the two family members, the one from the site that has a bigger sample size
dropids=['sub-NDARINVB3CU3GHK',   'sub-NDARINVZRABEXLZ',   'sub-NDARINVVH9GWJVF',   'sub-NDARINVC2ZLETPN',   'sub-NDARINVGFCRX7YW']

for i in dropids:
    ids.remove(i)
len(ids)

8884

mean motion > 0.25

In [19]:
#consort cbcl baseline
newid = cbcl[cbcl['meanFDj'] < 0.25].index 
cbcl = cbcl.loc[newid]
ids = newid.copy().to_list()
len(ids)

4111

In [20]:
8884 - 4111

4773

at least 10min of good quality data

In [21]:
pconn_ids = pd.read_csv(f'{main_dir}/cpm/abcd_dcan/10min/connectomes/pconn_ids.csv', sep=',')
grab10min = pconn_ids['participant_id'].to_list()
grab10min = ['sub-' + x for x in grab10min]
original_order = grab10min
len(grab10min)

5955

In [22]:
filtered_indices = cbcl.index.intersection(grab10min)
final = cbcl.loc[filtered_indices]
ids = final.index.to_list()
len(ids)

3721

have to remove these ids because they got pulled back in from the pconn_ids

In [23]:
# drop ids of one of the two family members, the one from the site that has a bigger sample size
dropids=['sub-NDARINVB3CU3GHK', #family   
         'sub-NDARINVZRABEXLZ', #family   
         'sub-NDARINVVH9GWJVF', #family   
         'sub-NDARINVC2ZLETPN', #family   
         'sub-NDARINVGFCRX7YW', #family   
         'sub-NDARINVP2G0PXCM', #is in Collection 3165 but miss all info from ABCD. in CONSORT figure count towards: missing cbcl base  
        ] 

# Check if elements of list1 are in list2
for i in dropids:
    if i in ids:  
        print(f"removing {i}")  
        ids.remove(i)
len(ids)

removing sub-NDARINVB3CU3GHK
removing sub-NDARINVC2ZLETPN
removing sub-NDARINVGFCRX7YW


3718

get all behave data

In [1]:
used = pd.read_csv(f'{main_dir}/behave_inputs/abcd_dcan/abcd_dcan_allsubjs_10min.csv', sep = ",")
used = used.set_index('participant_id')
used = used.loc[ids]
used

NameError: name 'pd' is not defined

# Grab other info for demographic table

In [25]:
demo = pd.read_csv(f'{main_dir}/data/abcd/ABCDStudyNDA/pdem02.txt', sep = "\t", skiprows=[1])
demo['participant_id'] = demo['src_subject_id'].str.replace('_','').str.replace('NDAR','sub-NDAR')
demo = demo.set_index('participant_id')
demo.head(1)

,collection_id,pdem02_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,demoi_p_select_language___1,demo_prim,...,demo_yrs_1,demo_yrs_2,eventname,demo_yrs_2a_2,demo_yrs_2b_2,demo_yrs_2_no_display___1,demo_race_a_p___0,demo_yrs_2_2,collection_title,study_cohort_name
participant_id,,,,,,,,,,,,,,,,,,,,,
sub-NDARINV00LH735Y,2573,28284,47381,NDAR_INV00LH735Y,NDAR_INV00LH735Y,01/29/2018,109,M,NaN,1,...,3.0,4.0,baseline_year_1_arm_1,NaN,NaN,1,0,NaN,Adolescent Brain Cognitive Development Study (...,ABCD 4.0 Data Release


## remap some questions
### https://nda.nih.gov/data-structure/pdem02

- demo_race_a_p___xxxx What race do you consider the child to be? Please check all that apply. ¿De qué raza considera que es el/la niño(a)? Seleccione 1 o más de estas categorías

In [26]:
race_dict = {'10':'White',
             '11':'Black_African_American',
             '12':'American_Indian_Native_American',
             '13':'Alaska_Native',
             '14':'Native_Hawaiian',
             '15':'Guamanian',
             '16':'Samoan',
             '17':'Other_Pacific_Islander',
             '18':'Asian_Indian',
             '19':'Chinese',
             '20':'Filipino',
             '21':'Japanese',
             '22':'Korean',
             '23':'Vietnamese',
             '24':'Other_Asian',
             '25':'Other_Race',
             '77':'Refuse_Answer',
             '99':'Dont_Know'
            }

# Function to rename columns based on the race_dict
def rename_columns(col):
    if 'demo_race_a_p___' in col:  # Only apply renaming to columns that contain this pattern
        for key, value in race_dict.items():
            if key in col:
                return col.replace(key, value)
    return col

# Rename columns selectively
demo.columns = [rename_columns(col) for col in demo.columns]

- demo_ethn_v2	Integer	Recommended	Do you consider the child Hispanic/Latino/Latina? ¿Considera que el/la niño(a) es hispano(a) o latino(a)?	1 ; 2 ; 777 ; 999	1 = Yes SÃ­; 2 = No No; 777 = Refuse to answer Niego contestar; 999 = Don't know No lo sÃ©

In [27]:
demo['demo_ethn_v2'].unique()

array([  1.,   2., 999., 777.,  nan])

In [28]:
ethnic_latino_dict = {
    1.0: 'yes',
    2.0: 'no',
    999.0: 'dont know',
    777.0: 'refuse to answer'
}

# Replace values in 'column_name' using the mapping dictionary
demo['ethnic_group_latino'] = demo['demo_ethn_v2'].replace(ethnic_latino_dict)

- demo_ethn2_v2	Integer	Recommended	Please choose the group that best represents the child's Hispanic origin or ancestry Elija el grupo que mejor represente el origen o la ascendencia hispana del niño o la niña.	10 ; 12 ; 13 ; 14 ; 15 ; 18 ; 19 ; 20 ; 40 ; 41 ; 777 ; 999	10 = Puerto Rican PuertorriqueÃ±o ; 12 = Dominican (Republic) Dominicano ; 13 = Mexican Mexicano ; 14 = Mexican American MÃ©xico-estadounidense ; 15 = Chicano; 18 = Cuban Cubano ; 19 = Cuban American Cubano-estadounidense ; 20 = Central or South American Centroamericano o sudamericano ; 40 = Other Latin American Otro origen latinoamericano ; 41 = Other Hispanic Otro origen hispano ; 777 = Refused to answer Prefiero no responder ; 999 = Don't know No lo sÃ©
demo_origin_v2	Integer	Recommended	In which country was the child born? ¿En qué país nació el/la niño(a)?	1::198; 777; 999	

In [29]:
ethnic_group_dict = {10.0 : 'Puerto_Rican',
                     12.0 : 'Dominican_Republic', 
                     13.0 : 'Mexican', 
                     14.0 : 'Mexican_American', 
                     15.0 : 'Chicano',
                     18.0 : 'Cuban',
                     19.0 : 'Cuban_American', 
                     20.0 : 'Central_or_South_American', 
                     40.0 : 'Other_Latin_American', 
                     41.0 : 'Other_Hispanic', 
                     777.0 : 'Refused_answer', 
                     999.0 : 'Dont_know'
                    }

demo['ethnic_group_which'] = demo['demo_ethn2_v2'].replace(ethnic_group_dict)

In [2]:
demo = demo[['ethnic_group_latino','ethnic_group_which'] + demo.columns[demo.columns.str.contains('demo_race')].to_list()].loc[final.index]
demo

NameError: name 'demo' is not defined

In [47]:
grab_q = ['sex_x','interview_age_bl','interview_age_y1','cbcl_scr_syn_anxdep_r_bl','cbcl_scr_syn_anxdep_r_y1','cbcl_scr_syn_anxdep_t_bl','cbcl_scr_syn_anxdep_t_y1','scanner_manufacturer', 'scanner_model','meanFDj','site',]

final = pd.merge(used[grab_q], demo, on='participant_id')

In [48]:
final.isna().sum()

sex_x                                                 0
interview_age_bl                                      0
interview_age_y1                                      0
cbcl_scr_syn_anxdep_r_bl                              0
cbcl_scr_syn_anxdep_r_y1                              0
cbcl_scr_syn_anxdep_t_bl                              0
cbcl_scr_syn_anxdep_t_y1                              0
scanner_manufacturer                                  0
scanner_model                                         0
meanFDj                                               0
site                                                  0
ethnic_group_latino                                   5
ethnic_group_which                                 3117
demo_race_a_p___White                                 0
demo_race_a_p___Black_African_American                0
demo_race_a_p___American_Indian_Native_American       0
demo_race_a_p___Alaska_Native                         0
demo_race_a_p___Native_Hawaiian                 

# Save to file ABCD

In [49]:
final.to_csv(f'{main_dir}/behave_inputs/consort_final_sample_abcd_n{len(final.index)}.csv', sep = ',')

# Double check
these are the ids that the CPM scripts used. double checking that the final set used by CPM and final set generated by applying exclusion criteria (but starting from original ABCD data) match

In [34]:
cpm = pd.read_csv(f'{main_dir}/cpm/abcd_dcan/10min/outputs/cbcl_scr_syn_anxdep_t_y1_gordon352_2fold_pearson_partialcovar_cbcl_scr_syn_anxdep_t_blmeanFDjsex_xinterview_age_bl_p0.001_FD0.25_allsubjs/data_used.csv')
cpm = cpm.set_index('participant_id')

In [35]:
inc = cpm[cpm['excluded'] == 0].index.to_list() # included 
exc = cpm[cpm['excluded'] == 1].index.to_list() # excluded #.dropna(how='any')
print(f'included = {len(inc)}')
print(f'excluded = {len(exc)}')

included = 3718
excluded = 2237


In [36]:
list1 = final.index.to_list()
list2 = inc

# Elements unique to list1
unique_to_list1 = list(set(list1) - set(list2))

# Elements unique to list2
unique_to_list2 = list(set(list2) - set(list1))

# All elements that are different between the two lists
different_elements = list(set(list1).symmetric_difference(set(list2)))

print("Unique to list1:", unique_to_list1)
print("Unique to list2:", unique_to_list2)
print("All different elements:", different_elements)

Unique to list1: []
Unique to list2: []
All different elements: []


# BANDA

In [3]:
beh = pd.read_csv(f'{main_dir}/behave_inputs/banda/banda_allsubjs_washu.csv', sep = ",")
beh['participant_id'] = beh['src_subject_id']
merge_cols = ['participant_id','subjectkey','sex'] #cols that will be used to merged files
beh = beh.set_index(merge_cols)
ids = beh.index.to_list()
beh

NameError: name 'pd' is not defined

In [38]:
len(beh)

202

rcads

In [39]:
#consort rcads baseline
newid = beh[beh['rcads01_rcads_tot_t_T1'].notna()].index 
beh = beh.loc[newid]
print(len(beh))

200


In [40]:
#consort rcads baseline
newid = beh[beh['rcads01_rcads_tot_t_T3'].notna()].index 
beh = beh.loc[newid]
print(len(beh))

154


In [41]:
200-154

46

drop ids

In [42]:
#consort cbcl baseline
newid = beh[beh['Movement_RelativeRMS_mean_T1'] < 0.25].index 
beh = beh.loc[newid]
ids = newid.copy().to_list()
len(ids)

150

In [43]:
beh[['sex01_T1','interview_age_T1','interview_age_T3','ndar_subject01_race_T1','ndar_subject01_ethnic_group_T1','rcads01_rcads_tot_T1','rcads01_rcads_tot_T3','rcads01_rcads_tot_t_T1','rcads01_rcads_tot_t_T3','Movement_RelativeRMS_mean_T1']].isna().sum()

sex01_T1                          0
interview_age_T1                  0
interview_age_T3                  0
ndar_subject01_race_T1            0
ndar_subject01_ethnic_group_T1    0
rcads01_rcads_tot_T1              0
rcads01_rcads_tot_T3              0
rcads01_rcads_tot_t_T1            0
rcads01_rcads_tot_t_T3            0
Movement_RelativeRMS_mean_T1      0
dtype: int64

# Save to file ABCD

In [46]:
beh.to_csv(f'{main_dir}/behave_inputs/consort_final_sample_banda_n{len(beh.index)}.csv', sep = ',')